In [1]:
import pandas as pd
import numpy as np
import email
import os
from email.parser import Parser
import re

In [2]:
# extract the body of email
def walkMsg(msg):
    """
    Input: string of email
    Output: body of email
    """
    for part in msg.walk():
        if part.get_content_type() == "multipart/alternative":
            continue
        yield part.get_payload(decode=1)

In [60]:
# extract the information of email
# still need some works to get number of cc and bcc
def info_of_email(path,List_of_files,classification):
    """
    Input: path, list of files name, classification(spam or ham)
    Ourput: a list with info of email
    """
    parser = Parser()
    info = []
    for files in List_of_files:
        emailText =  open(path + files, 'rb').read()
        email = parser.parsestr(emailText)
        if email.get('cc') == None:
            no_cc = 0
        else:
            no_cc = len(email.get('cc'))
        if email.get("bcc") == None:
            no_bcc = 0
        else:
            no_bcc = len(email.get('bcc'))
        Date = email.get("Date")
        special_chars = re.compile(r"\t\n\r")
        try:
            Date = re.split(':|,| ',Date)
            info.append([Date[4], Date[3], Date[2], Date[0], Date[5], email.get("From"), 
                         email.get('To'), no_cc, no_bcc, 
                         str(email.get("Content-Type")).split(";")[0],
                         email.get('subject'), 
                         " ".join([i for i in walkMsg(email) if i != None]).replace("\n"," ").replace("\t"," ").replace("r"," "),
                         classification])
        except:
            info.append([[], [], [], [], [], email.get("From"), 
                         email.get('To'), no_cc, no_bcc, 
                         str(email.get("Content-Type")).split(";")[0],
                         email.get('subject'), 
                         " ".join([i for i in walkMsg(email) if i != None]).replace("\n"," ").replace("\t"," ").replace("r"," "),
                         classification])
    return info

In [61]:
listing = os.listdir('./Spam/')
listing = [i for i in listing if i[-5:] != "Store"]
email_df = pd.DataFrame(info_of_email('./Spam/', listing, 'spam'),
                       columns=['Year','Month','Day','Weekday',"Hour",
                                'From','To','cc','bcc','Content_type','Subjects','Content','Classification'])

In [62]:
email_df.tail()

,Year,Month,Day,Weekday,Hour,From,To,cc,bcc,Content_type,Subjects,Content,Classification
31096,2002,Dec,3,Tue,18,"""IQ - TBA"" <tba@insiq.us>",<yyyy@spamassassin.taint.org>,0,0,multipart/alternative,Preferred Non-Smoker Rates for Smokers,P efe ed Non-Smoke Just what the doct...,spam
31097,2003,Jul,20,Sun,16,Mike <raye@yahoo.lv>,Mailing.List@user2.pro-ns.net,0,0,text/plain,"How to get 10,000 FREE hits per day to any web...","Dea Subsc ibe , If I could show you a way to...",spam
31098,2020,Aug,05,Wed,04,"""Mr. Clean"" <cweqx@dialix.oz.au>",<Undisclosed.Recipients@webnote.net>,0,0,text/plain,Cannabis Difference,****Mid-Summe Custome App eciation SALE!****...,spam
31099,2002,Dec,04,Wed,06,"""wilsonkamela400@netscape.net"" <wilsonkamela50...",ilug@linux.ie,0,0,text/plain,[ILUG] WILSON KAMELA,ATTN:SIR/MADAN ST...,spam
31100,2005,Jul,05,Tue,09,"""Chia Patterson"" <pkrebehenne@businessesinport...",projecthoneypot@projecthoneypot.org,0,0,text/html,Just to her...,"<html> <body alink=""blue"" bgcolo =""white"" link...",spam


In [43]:
email_df.shape

(31101, 13)

In [45]:
sender = email_df.iloc[0,5]
sender

'"Bertha " <Denny@mailbox.sk>'

In [64]:
re.findall('.+@.+',sender)[0]

'"Bertha " <Denny@mailbox.sk>'

In [63]:
email_df.iloc[:20,5]

0                          "Bertha " <Denny@mailbox.sk>
1     "LZEDEMCNODQBEZQBOV@encryption.com" <Opal.Cham...
2                     Connie Wong <tatypin@huhmail.com>
3           shark@promotions.com <shark@promotions.com>
4     "Important! Search Engine Alert!" <rockwelldat...
5                 "Louis Henrietta" <ndqpgk4ug@ioc.net>
6                       "Infor Info" <infor_123@ozu.es>
7             "Roberto Lange" <xaotljzjaho@hotmail.com>
8             "Harriet Mcgee" <therein2@pinghetang.com>
9       "Brian_loebig" <Jikkloscapegoat@africamail.com>
10    "Corey Hernandez" <ifekyc@highperformancemail....
11                     "Steven" <gatewnq@optonline.net>
12                             "James" <gatewgt@rr.com>
13                                blh0002@aarhusmail.dk
14                    "Bruce" <summerrelishes@mail.com>
15                           rollinsbc@to-pornostarz.de
16    "Valarie Richardson" <valarierichardson_to@hel...
17          investment consultant <zdywib@hushma